In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install imutils

In [5]:
import argparse

In [7]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0,(300,300),(104.0, 177.0, 123.0))  
    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        
        confidence = detections[0,0,i,2]
        
    if len(faces) > 0:
        
        
        
        faces = np.array(faces,types="float32")
        preds = maskNet.predict(faces, batch_size=32)
    
    
    return (locs, preds)

ap = argparse.ArgumentParser()
ap.add_argument("-f","--face", type=str,
     default="face_detector",
     help="path to face detector model directory")
ap.add_argument("-m","--model", type=str,
     default="mask_detector.model",
     help="path to trained face mask detector model")
ap.add_argument("-c", "--confidence",type=float, default=0.5,
     help="minimum probability to filter weak detections")
args = vars(ap.parse_args())


print("[INFO] loading face detector model...")     
prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("mask_detector.model")

print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

while True:
    
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    (locs,preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
for (box,pred) in zip(locs, preds):
        
        (startX,startY, endX ,endY)= box
        (Mask, withoutMask) = pred
          
        label = "Mask" if Mask > witoutMask else "No Mask"
        color = (0,255, 0) if label =="Mask"  else (0,0,225)
        #frame
        
        
        if (label == "Mask"):
            
            cv2.putText(frame, "Mask: You are allowed", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2) 
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        elif(label=="No Mask"):
            lab ="No Mask: You are not allowed"
            cv2.putText(frame,lab,(startX,startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2) 
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            
    cv2.imshow("Frame",frame)
    key = cv2.waitkey(1) & 0xFF
    
    if key == ord("q"):
        break
                
cv2.destroyAllWindows()

vs.release()

  
     


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 73)